In [18]:
from typing import Optional
from datetime import datetime
from pydantic import BaseModel, Field
from openai import OpenAI
import requests
import logging
import json

In [19]:
client = client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

model = "llama3.2:3b"

In [20]:
class check_prompt(BaseModel):
    """First LLM call: Extract basic accomodation information"""

    # description: str = Field(description="Raw description of the event")
    is_accomodation_search: bool = Field(
        description="Is the text a query related to finding an accomodation?"
    )


class EventDetails(BaseModel):
    """Second LLM call: Parse specific event details"""

    city: str = Field(description="City in which the accomodation should be booked")
    start_date: str = Field(
        description="Date and time of checking into the accomodation. Strictly use 'yyyy-mm-dd' date format for this value. Example is 2025-03-25"
    )
    end_date: str = Field(
        description="Date and time of checking out of the accomodation. Strictly use 'yyyy-mm-dd' date format for this value. Example is 2025-03-25"
    )

In [21]:
def check_accomodation_request(input: str):
    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "You are an assistant for finding accomodation. \
                    You are allowed to answer only to queries related to accomodation search. \
                    In case of other requests, you must decline. \
                    In this context, Analyze if the text describes a query for finding accomodation. \
                    Say yes if the text describes a query for finding accomodation, else say no",
            },
            {"role": "user", "content": input},
        ],
        response_format=check_prompt
    )

    return completion.choices[0].message.parsed.is_accomodation_search


def extract_stay_details(input:str):
    completion = client.beta.chat.completions.parse(
        model=model,
        messages=[
            {
                "role": "system",
                "content": f"You are an assistant for finding accomodation. \
                    Extract the name of the city in which the user wishes to stay. If city is not found, strictly use the placeholder [None].\
                    Extract the check-in date. Check-in date is date on which the user will start staying in the accomodation. Strictly use 'yyyy-mm-dd' date format. If check-in date is not found, strictly use the placeholder [None].\
                    Extract the check-out date. Check-out date is date on which the user will leave the accomodation.  Strictly use 'yyyy-mm-dd' date format. If check-out date is not found, strictly use the placeholder [None]. \
                    Do not make any assumptions for the city, check-in date and check-out date.",
            },
            {"role": "user", "content": input},
        ],
        response_format=EventDetails
    )

    return completion

In [22]:
def process_accomodation_search(input: str):
    is_accomodation_request = check_accomodation_request(input)

    if not is_accomodation_request:
        return None
    else:
        stay_details = extract_stay_details(input)
        return stay_details

In [23]:
user_input = "Suggest me some hotels in Bengaluru for staying from 27th March 2025 and 31st March 2025"
# user_input = "Suggest me some hotels in Bengaluru"
# user_input = "Write a poem"

In [24]:
result = process_accomodation_search(user_input)

if not result:
    print("This is not an accomodation search request!")
else:
    pass

In [25]:
place_of_stay = result.choices[0].message.parsed.city
check_in_date = result.choices[0].message.parsed.start_date
check_out_date = result.choices[0].message.parsed.end_date

In [26]:
place_of_stay, check_in_date, check_out_date

('Bengaluru', '2025-03-27', '2025-03-31')

In [27]:
def get_weather_forecast(latitude, longitude, start_date, end_date):
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&hourly=temperature_2m,relative_humidity_2m,rain&start_date={start_date}&end_date={end_date}"
    )
    data = response.json()
    return data

In [28]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather_forecast",
            "description": "Get the forecast for temperature in Celsius, relative humidity in percentage and rain in mm for provided coordinates between the start date and end date.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                    "start_date": {"type": "string"},
                    "end_date": {"type": "string"}
                },
                "required": ["latitude", "longitude", "start_date", "end_date"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

In [107]:
system_prompt = "You are a helpful weather assistant. \
                You can determine the latitude and longitude based on the name of the city/place \
                You strictly use 'yyyy-mm-dd' format for dates. \
                Based on the hourly temperature, relative humidity and rain data, you can provide a brief summary on the weather in natural language."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"What is the weather forecast for {place_of_stay} between {check_in_date} and {check_out_date}?"},
]

completion = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
)

In [108]:
completion.model_dump()

{'id': 'chatcmpl-256',
 'choices': [{'finish_reason': 'tool_calls',
   'index': 0,
   'logprobs': None,
   'message': {'content': '',
    'refusal': None,
    'role': 'assistant',
    'annotations': None,
    'audio': None,
    'function_call': None,
    'tool_calls': [{'id': 'call_vkkv1h67',
      'function': {'arguments': '{"end_date":"2025-03-31","latitude":"12.9848","longitude":"77.5621","start_date":"2025-03-27"}',
       'name': 'get_weather_forecast'},
      'type': 'function',
      'index': 0}]}}],
 'created': 1742904172,
 'model': 'llama3.2:3b',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_ollama',
 'usage': {'completion_tokens': 55,
  'prompt_tokens': 297,
  'total_tokens': 352,
  'completion_tokens_details': None,
  'prompt_tokens_details': None}}

In [109]:
def call_function(name, args):
    if name == "get_weather_forecast":
        return get_weather_forecast(**args)

In [110]:

for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    messages.append(completion.choices[0].message)

    result = call_function(name, args)
    messages.append(
        {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result['hourly'])}
    )

In [111]:

class WeatherResponse(BaseModel):
    response: str = Field(
        description="A summary on the weather in natural language based on the weather forcaset data")

In [116]:
completion_2 = client.beta.chat.completions.parse(
    model=model,
    messages=messages,
    tools=tools,
    response_format=WeatherResponse,
)

In [117]:
completion_2.model_dump()

{'id': 'chatcmpl-553',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '{\n"response": "Invalid data for plotting the time series of precipitation and humidity"\n}',
    'refusal': None,
    'role': 'assistant',
    'annotations': None,
    'audio': None,
    'function_call': None,
    'tool_calls': None,
    'parsed': {'response': 'Invalid data for plotting the time series of precipitation and humidity'}}}],
 'created': 1742904384,
 'model': 'llama3.2:3b',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'fp_ollama',
 'usage': {'completion_tokens': 22,
  'prompt_tokens': 2048,
  'total_tokens': 2070,
  'completion_tokens_details': None,
  'prompt_tokens_details': None}}

In [75]:
messages

[{'role': 'system',
  'content': "You are a helpful weather assistant.                 Decide the latitude and longitude based on the name of the city/place                 Strictly use 'yyyy-mm-dd' format for dates.                 Based on the dictionary containing hourly temperature, relative humidity and rain data, provide a concise summary on the weather in natural language.                 Do not make any assumptions. Strictly base your response only on the available data"},
 {'role': 'user',
  'content': 'What is the weather forecast for Bengaluru between 2025-03-27 and 2025-03-31?'},
 ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_nsn9eply', function=Function(arguments='{"end_date":"2025-03-31","latitude":13,"longitude":77,"start_date":"2025-03-27"}', name='get_weather_forecast'), type='function', index=0)]),
 {'role': 'tool',
  'tool_call_id': 'call_nsn9eply'